# 注意！ 作成中です👽

# xlwings と openpyxl  

xlwings と openpyxl はともに Excel ファイルを操作するための代表的な Python パッケージです。両者とも Anaconda に同梱されています。  
簡単に比較すると、  

|[xlwings](https://www.xlwings.org/)|[openpyxl](https://openpyxl.readthedocs.io/)| 
|-----------------------------------|--------------------------------------------|
|Excel アプリを通して Excel ファイルを取り扱う|Excel ファイルを直接操作する|
|Excel アプリのインストールが必要|Excel アプリ不要で利用できる|                             
|処理が遅い|速い|
|Excel のバグも考慮しなければならない|Excel のバグの影響をうけない|
|制限は Excel に準じる|ブックを超えたシートのコピーなどが出来ない|
|VBA に近い|VBA とはかなり異なる|

このような違いがあります。  

コードの実行とともに Excel が動くので、xlwings のほうがコーディングやデバッグはしやすいと感じます。  

[xlwings - 翔べ！Excel — xlwings dev ドキュメント](https://docs.xlwings.org/ja/latest/)  
[openpyxl - A Python library to read/write Excel 2010 xlsx/xlsm files — openpyxl 3.0.3 documentation](https://openpyxl.readthedocs.io/en/stable/)  

## xlwings

In [102]:
# xlwings のインポート
import xlwings as xw

In [103]:
# 新規 Book を作成 (Excel が起動します)
book_xw = xw.Book()
print(type(book_xw))

<class 'xlwings.main.Book'>


In [104]:
# 作成したファイルを名前を付けて保存
book_xw.save("book_xw.xlsx")

In [105]:
s = book_xw.sheets.active

In [106]:
rng = s.range("A1:J2")

In [107]:
rng.value = [[c for c in row] for row in ["ABCDEFGHIJ", "1234567890"]]  # リスト内法表記を利用

In [76]:
# 最後に Excel 自体を終了させる必要があるため 
# Excel アプリを取得しておく
app_xw = book_xw.app
print(type(app_xw))

<class 'xlwings.main.App'>


In [77]:
# Sheets を取得
# VBA の Worksheets コレクションに相当
sheets_xw = book_xw.sheets
print(type(sheets_xw))

<class 'xlwings.main.Sheets'>


In [78]:
# Sheets へのインデックス指定で Sheet を取得
# [] でのインデックス指定は 0 から始まる (Python 風)
# () でのインデックス指定は 1 から始まる (VBA 風)
print(type(sheets_xw[0]))
print(sheets_xw[0].name)
print(sheets_xw(1).name)
# 名前を指定しての取得も可能
print(sheets_xw["Sheet1"].name)

<class 'xlwings.main.Sheet'>
Sheet1
Sheet1
Sheet1


In [79]:
# 新しいシートを追加
# 追加されたシートが自動的にアクティブになる
sheets_xw.add("シート2", after="Sheet1")
print("現在のシート数は {} です。".format(sheets_xw.count))
print("現在アクティブなシートは {} です。".format(sheets_xw.active.name))

現在のシート数は 2 です。
現在アクティブなシートは シート2 です。


In [80]:
print(sheets_xw[1].name)          # Python 風
print(sheets_xw(2).name)          # VBA 風
print(sheets_xw["シート2"].name)  # VBA 風

シート2
シート2
シート2


In [81]:
# Sheet1 をアクティブにする
sheets_xw("Sheet1").activate()
print("現在アクティブなシートは {} です。".format(sheets_xw.active.name))

現在アクティブなシートは Sheet1 です。


In [82]:
# Sheet1 → シート1 にリネーム
s1_xw = sheets_xw("Sheet1")
s1_xw.name = "シート1"
print("現在アクティブなシートは {} です。".format(sheets_xw.active.name))

現在アクティブなシートは シート1 です。


In [83]:
# Range を取得してアクセス
s1_xw[0, 0].value = "A1"          # Python 風
s1_xw.range("B1").value = "B1"    # VBA 風
s1_xw.range((1, 3)).value = "C1"  # VBA 風

In [84]:
# 複数のセル範囲に対する処理
# セルの背景色 RGB をタプルで渡して指定
s1_xw.range("A2:C2").color = (255, 0, 0)  

In [85]:
# Sheet.range((from), (to)) で VBA 風のインデックスを渡
# セルの背景色をクラスで定義された定数で指定
s1_xw.range((3, 1), (4, 4)).color = xw.constants.RgbColor.rgbBlue 

xlwings で利用できるその他の定数の一覧は以下を参照ください。  
[xlwings/constants.py at master · xlwings/xlwings](https://github.com/xlwings/xlwings/blob/master/xlwings/constants.py)  

In [86]:
# セルの背景色をなくす
s1_xw.range("3:3").color = None

In [88]:
sheets_xw.add("sheet5", before=1)

<Sheet [book_xw.xlsx]sheet5>

In [56]:
# Sheet.cells() は range を返す
s1_xw.cells(4, 1).value = "A4"
print(type(s1_xw.cells(4, 1)))

<class 'xlwings.main.Range'>


In [57]:
# 式の入力
s1_xw.cells(4, 2).formula = "=\"B\" & right(A4, 1)"  # 文字列中のダブルクォーテーションは \ でエスケープ

In [58]:
# copy を使ってセルのコピー
# 相対セル参照部分も移動する
s1_xw.range((4, 2)).copy(destination=s1_xw.range((4, 3)))

In [59]:
# この方法では相対セル参照であっても元の数式のとおり
s1_xw.range((4, 4)).formula = s1_xw.range((4, 2)).formula

In [60]:
# シート2を表示 (アクティブに)
s2_xw = sheets_xw("シート2")
s2_xw.activate()
print("現在アクティブなシートは {} です。".format(sheets_xw.active.name))

現在アクティブなシートは シート2 です。


In [61]:
# グラフを描くためのデータを作成
import numpy as np

x = np.linspace(-5, 5, 101)
y_sin = np.sin(x)
y_cos = np.cos(x)

In [62]:
# データをシートに入力
s2_xw.range("A1").value = x
s2_xw.range("A2").value = y_sin
s2_xw.range("A3").value = y_cos

In [63]:
# データの個数を確認
print(len(x))

101


In [64]:
# 使用されている最大の行を取得
s2_xw.used_range.last_cell.row

3

In [65]:
# 使用されている最大の列を取得
# データの個数と一致
s2_xw.used_range.last_cell.column

101

In [66]:
# 取得されているのは Range
last_cell = s2_xw.used_range.last_cell
print(type(last_cell))

<class 'xlwings.main.Range'>


In [67]:
# Range.get_address() を使うと A1 形式のアドレスが返る
s2_xw.used_range.last_cell.get_address()

'$CW$3'

In [68]:
# データをもとに Chart (グラフ) を作成
chart_xw = s2_xw.charts.add()
print(type(chart_xw))

<class 'xlwings.main.Chart'>


In [69]:
chart_xw.set_source_data(s2_xw.range("A2:{}".format(last_cell.get_address())))
chart_xw.chart_type = 'line'
chart_xw.top = 100  #　グラフの上下方向の位置

In [70]:
# api を使って Excel ネイティブのオブジェクトを操作
# ここでは x 軸の目盛りを変換
chart_xw.api[1].SeriesCollection(1).XValues = s2_xw.api.Range("A1:CW1")

In [71]:
# ここでは凡例ラベルを変更
names = ("y_sin", "y_cos")
for s, n in zip(chart_xw.api[1].SeriesCollection(), names):
    s.XValues = s2_xw.api.Range("A1:CW1")
    s.Name = n

注意: Chart.api は要素が 2 つのタプルを返します。  
2 番目の要素 (インデックス指定は `[1]`) を指定することでうまく操作できるようです。(理由はよくわかりません💦)  

[python - set chart name in Xlwings - Stack Overflow](https://stackoverflow.com/questions/44910566/set-chart-name-in-xlwings?rq=1)  

In [101]:
# 上書き保存して終了
book_xw.save()
book_xw.close()
app_xw.quit()  # ← これがないと Excel のプロセスが残ってしまう

## openpyxl

In [31]:
# openpyxl のインポート
import openpyxl as px

In [32]:
# 新規 Workbook の作成
book_px = px.Workbook()
print(type(book_px))

<class 'openpyxl.workbook.workbook.Workbook'>


In [33]:
# WorkBook 内に存在しているシート名のリスト
book_px.sheetnames

['Sheet']

In [34]:
# Workbook 内に存在している Worksheet のリスト
sheets_px = book_px.worksheets
print(type(sheets_px))

<class 'list'>


In [35]:
# Worksheet へのアクセスとシート名の表示
print(sheets_px[0].title)
print(book_px["Sheet"].title)
print(type(sheets_px[0]))

Sheet
Sheet
<class 'openpyxl.worksheet.worksheet.Worksheet'>


In [36]:
# シート名の変更
s1_px = sheets_px[0]
s1_px.title = "シート1"
book_px.sheetnames

['シート1']

In [39]:
s1_px["A1"].value = 5
s1_px["A2"].value = 15
s1_px["A3"] = "=A1+A2"
s1_px.cell(row=4, column=1, value=101)
s1_px.cell(5, 1).value = 105
print(type(s1_px["A1"]))

<class 'openpyxl.cell.cell.Cell'>


In [40]:
book_px.save("book_px.xlsx")

In [7]:
# 新しい Worksheet の追加
s2_px = book_px.create_sheet(title="シート2", index=1)

In [8]:
book_px.sheetnames

['シート1', 'シート2']

In [17]:
rng = s2_px["B5":"C6"]
print(rng)
print(type(rng))

((<Cell 'シート2'.B5>, <Cell 'シート2'.C5>), (<Cell 'シート2'.B6>, <Cell 'シート2'.C6>))
<class 'tuple'>


In [13]:
for row in rng:
    for c in row:
        print(c.coordinate)

B5
C5
B6
C6


In [16]:
s1_px.append([1, 2, 3, 4])  # appnd するたびに一行ずつ書き込み
book_px.save("book_px.xlsx")

In [118]:
s2_px["A5":].value = x
s2_px.cell(6, len(y_sin)).value = y_sin

IndexError: slice('A5', None, None) is not a valid coordinate or range

In [11]:
book_px.save("book_px.xlsx")

In [18]:
import openpyxl as px

In [19]:
book = px.Workbook()
book

In [20]:
sheets = book.worksheets
sheets

[<Worksheet "Sheet">]

In [21]:
sheet = sheets[0]
sheet

<Worksheet "Sheet">

In [22]:
sheet.cell(1, 1)
sheet.append([1])
sheet

<Worksheet "Sheet">

In [23]:
book.worksheets.append(1)

In [24]:
book.worksheets

[<Worksheet "Sheet">]

In [26]:
import pandas as pd
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows

wb = px.Workbook()
ws = wb.active

df = pd.DataFrame(np.arange(12).reshape(3, 4))

for r in dataframe_to_rows(df, index=True, header=True):
    print(r)
    ws.append(r)
    
wb.save("pd_test.xlsx")

<generator object dataframe_to_rows at 0x000001F2EDA8AA48>
[None, 0, 1, 2, 3]
[None]
[0, 0, 1, 2, 3]
[1, 4, 5, 6, 7]
[2, 8, 9, 10, 11]


In [27]:
for cell in ws["A"] + ws["1"]:
    print(cell)
    cell.style = "Pandas"
    
wb.save("pd_test.xlsx")

<Cell 'Sheet'.A1>
<Cell 'Sheet'.A2>
<Cell 'Sheet'.A3>
<Cell 'Sheet'.A4>
<Cell 'Sheet'.A5>
<Cell 'Sheet'.A1>
<Cell 'Sheet'.B1>
<Cell 'Sheet'.C1>
<Cell 'Sheet'.D1>
<Cell 'Sheet'.E1>


In [30]:
ws["A"] + ws["1"]

(<Cell 'Sheet'.A1>,
 <Cell 'Sheet'.A2>,
 <Cell 'Sheet'.A3>,
 <Cell 'Sheet'.A4>,
 <Cell 'Sheet'.A5>,
 <Cell 'Sheet'.A1>,
 <Cell 'Sheet'.B1>,
 <Cell 'Sheet'.C1>,
 <Cell 'Sheet'.D1>,
 <Cell 'Sheet'.E1>)